In [ ]:
import os
import json
import numpy as np

# 클래스 매핑
class_mapping = {
    "stand": 0,
    "sit": 1,
    "walk": 2,
    "bow": 3,
    "fall": 4,
}

# 데이터 로드 함수
def load_data_from_folders(base_dir):
    for class_name, label in class_mapping.items():
        X = []
        data_size = 0
        folder_path = os.path.join(base_dir, class_name)
        if os.path.isdir(folder_path):  # 디렉토리인 경우만 처리
            for file in os.listdir(folder_path):
                if file.endswith('.json'):
                    file_path = os.path.join(folder_path, file)
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                        for frame in data:  # 각 프레임을 개별 데이터로 추가
                            data_size += 1
                            if data_size == 5000:
                                break;
                            # 첫 번째 좌표를 기준으로 상대 좌표 변환
                            x1, y1, z1 = frame[0], frame[1], frame[2]  # 기준 좌표 추출
                            # 변환 로직
                            for i in range(0, len(frame), 3):
                                  x, y, z = frame[i], frame[i+1], frame[i+2]  # 현재 좌표 추출
                                  X.append(x - x1)     # x 상대 좌표
                                  X.append(y - y1)     # y 상대 좌표
                                  X.append(z - z1)     # z 상대 좌표
                if data_size == 5000:
                    break;

        # JSON으로 저장
        #output_data = {"relative_coordinates": X}
        # 파일로 저장
        with open(f"{class_name}.json", "w") as json_file:
            json.dump(X, json_file, indent=4)

def test(base_dir):
      folder_path = os.path.join(base_dir, "stand")
      if os.path.isdir(folder_path):  # 디렉토리인 경우만 처리
          for file in os.listdir(folder_path):
              if file.endswith('.json'):
                  file_path = os.path.join(folder_path, file)
                  with open(file_path, 'r') as f:
                      data = json.load(f)

                      for frame in data:  # 각 프레임을 개별 데이터로 추가
                          break
              break

load_data_from_folders("/content/dataset")

In [ ]:
import json
import numpy as np

def augment_frame(frame, translation_range=0.02, scale_range=0.1, rotation_range=10, noise_std=0.005):
    """
    단일 프레임에 증강 기법 적용.
    """
    # (1) Translation: 일정 범위 내에서 좌표 이동
    translation = np.random.uniform(-translation_range, translation_range, size=3)
    frame_aug = np.array(frame).reshape(-1, 3) + translation  # [x, y, z] 변환

    # (2) Scaling: 크기 변환
    scale = 1 + np.random.uniform(-scale_range, scale_range)
    frame_aug *= scale

    # (3) Rotation: z축 기준 회전 (yaw)
    theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    frame_aug = np.dot(frame_aug, rotation_matrix.T)

    # (4) Noise: 좌표에 노이즈 추가
    frame_aug += np.random.normal(0, noise_std, frame_aug.shape)

    return frame_aug.flatten().tolist()

def augment_json_data(json_file, output_file, num_augmentations=5):
    """
    JSON 파일 데이터 증강.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    augmented_data = []
    for frame in data:
        #augmented_data.append(frame)  # 원본 데이터 추가
        for _ in range(num_augmentations):
            augmented_frame = augment_frame(frame)
            augmented_data.append(augmented_frame)

    with open(output_file, 'w') as f:
        json.dump(augmented_data, f, indent=4)

    print(f"Saved augmented data to {output_file}, total frames: {len(augmented_data)}")

# 예제 실행
file_name = "/content/stand.json"
augment_json_data(file_name, file_name, num_augmentations=5)


Saved augmented data to /content/stand.json, total frames: 25000


In [ ]:
import os
import json
import numpy as np

# 클래스 매핑
class_mapping = {
    "stand": 0,
    "sit": 1,
    "walk": 2,
    "bow": 3,
    "fall": 4,
}

def load_data_from_folders(base_dir):
    labeled_data = []  # 라벨링된 데이터를 저장할 리스트

    for class_name, label in class_mapping.items():
        folder_path = os.path.join(base_dir, class_name)
        data_size = 0  # 처리한 프레임 수를 카운트
        if os.path.isdir(folder_path):  # 디렉토리인 경우만 처리
            for file in os.listdir(folder_path):
                if file.endswith('.json'):
                    file_path = os.path.join(folder_path, file)
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                        for frame in data:  # 각 프레임을 개별 데이터로 추가
                            if data_size == 5000:
                                break
                            # 첫 번째 좌표를 기준으로 상대 좌표 변환
                            x1, y1, z1 = frame[0], frame[1], frame[2]  # 기준 좌표 추출
                            relative_frame = []  # 상대 좌표 변환된 프레임
                            for i in range(0, len(frame), 3):
                                x, y, z = frame[i], frame[i + 1], frame[i + 2]  # 현재 좌표 추출
                                relative_frame.extend([x - x1, y - y1, z - z1])  # 상대 좌표 추가
                            # 라벨 추가
                            labeled_frame = relative_frame + [label]
                            labeled_data.append(labeled_frame)
                            data_size += 1
                if data_size == 5000:
                    break

    # 라벨링된 데이터를 하나의 JSON 파일로 저장
    output_file = "labeled_dataset.json"
    with open(output_file, "w") as json_file:
        json.dump(labeled_data, json_file, indent=4)
    print(f"Saved labeled data to {output_file}, total frames: {len(labeled_data)}")

# 데이터 로드 함수 실행
load_data_from_folders("/content/dataset")


TypeError: 'float' object is not subscriptable